# Trust in Multi-Agent Systems
Welcome to our lesson on trust and explainability in AI! 

Before we get started, remember to run this cell to get everything set up correctly:

In [2]:
%load_ext autoreload
%autoreload 2
from utils import prep_browser, run_and_viz_optic
from pyperplan.pddl.parser import Parser
from pyperplan import grounding, planner

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Explainability
As discussed in lecture, we know that _explaining_ unexpected behaviors from our robot can increase trust. In this module, we'll see how this comes into play. Recall our human-aware planning problem from lecture:

## Human-Aware Planning
Recall the Human-Aware Planning problem described in [1](https://arxiv.org/pdf/2105.01220.pdf).

**Input:**

$\mathcal{M}^R$, the robot's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^R, \mathcal{I}^R, \mathcal{G}^R\rangle$, where $\mathcal{D}^R$ is the domain,  $\mathcal{I}^R$ is the initial state, and $\mathcal{G}^R$ is the goal state.

$\mathcal{M}^G$, the human's model of the environment and problem. Consists of the tuple $\langle\mathcal{D}^H, \mathcal{I}^H, \mathcal{G}^H\rangle$, where $\mathcal{D}^H$ is the domain,  $\mathcal{I}^H$ is the initial state, and $\mathcal{G}^H$ is the goal state.

**Output:**

A _plan_; that is, a sequence of robot actions that achieve the goal state but also meets the human's expectations. We call the degree to which the robot plan $\pi$ matches the human expectations $\pi^e$ the plan _explicability_, and we often model it as the _distance_ $\delta$ between $\pi^e$ and $\pi$:

$$
E(\pi) = -1 * \delta(\pi^e, \pi)
$$

A plan $\pi$ is _perfectly explicable_ if $E(\pi) = 0$. We often use the difference in costs between the two plans as our distance function, $\delta$.


## Problem 1: Modeling Incomplete Knowledge
Because we're so familiar with PDDL, we will model both of these problems in PDDL! Check out the references from `ps-03` if you've forgotten how to use it. **TODO: describe this stuff in more detail.**

We'll start with a simplified version of the [Wumpus World](http://users.cecs.anu.edu.au/~patrik/pddlman/wumpus.html). Here, our robot is trying to navigate in a 3x3 grid to pick up a block. The robot knows that its shortest path is unencumbered, but the human has no idea--they think that there's piles of trash on the shortest path to the robot.

![Wumpus World](res/wumpus-world.png)

We've provided you with a fully modeled version of the robot world in `pddl/robot-domain.pddl` and `pddl/robot-problem.pddl`. Your task is to modify `pddl/human-domain.pddl` and `pddl/human-problem.pddl` so it reflects the fact that the _human_ thinks that there is trash in squares `(0, 2)` and `(1, 2)`. Notice that none of our actions are `durative-actions`; we assume a unit cost for every action.

Instead of using Optic to run our plans, we'll use a Python PDDL Planner called [`pyperplan`](https://github.com/aibasel/pyperplan). Pyperplan is cool because it is extensible and contains a very clean implementation of some of the commmon search algorithms; if you're interested in the way planners work, definitely check out their codebase! Like Optic, however, Pyperplan only supports _positive preconditions._  

Note, too, that we cannot specify negative initial conditions. This is important with our `clear` predicate. If we want to say that a square is not `clear` at the start, we simply omit it from the list of clear squares. See `robot-problem.pddl` for a concrete example.

In [26]:
domain_file = 'pddl/robot-domain.pddl'
problem_file = 'pddl/robot-problem.pddl'

parser = Parser(domain_file, problem_file)
domain = parser.parse_domain()
problem = parser.parse_problem(domain)

task = grounding.ground(problem)

search_alg = planner.SEARCHES['astar']
heuristic = planner.HEURISTICS['hff'](task)

search_alg(task, heuristic)

[<Op (robot-move robot1 sq2-2 sq1-2)>, <Op (robot-move robot1 sq1-2 sq0-2)>]

In [27]:
domain_file = 'pddl/human-domain.pddl'
problem_file = 'pddl/human-problem.pddl'

parser = Parser(domain_file, problem_file)
domain = parser.parse_domain()
problem = parser.parse_problem(domain)

task = grounding.ground(problem)

search_alg = planner.SEARCHES['astar']
heuristic = planner.HEURISTICS['hff'](task)

search_alg(task, heuristic)

[<Op (robot-move robot1 sq2-2 sq2-1)>,
 <Op (robot-move robot1 sq2-1 sq2-0)>,
 <Op (robot-move robot1 sq2-0 sq1-0)>,
 <Op (robot-move robot1 sq1-0 sq0-0)>,
 <Op (robot-move robot1 sq0-0 sq0-1)>,
 <Op (robot-move robot1 sq0-1 sq0-2)>]

## Problem 2: Thing Explainer

If you were the human in this human-robot team, you'd probably be concerned if your robot started moving towards where you thought was a gigantic pile of trash! This corresponds to a _decrease in trust_. As discussed in lecture, there are several ways to deal with these trust decreases:

TODO: enumerate the ways trust can decrease

Today, we'll be focusing on _explaining_; that is, telling our robot it's probably a good idea to explain its actions when it takes unexpected actions. We'll accomplish this by modifying the PDDL plan with an `explain` action that has some cost. **TODO: cost allocation**